# Non-Parametric Tests

# 1. Assumptions in hypothesis testing

<b>1.1 Common assumptions of hypothesis tests</b>

Hypothesis tests make assumptions about the dataset that they are testing, and the conclusions you draw from the test results are only valid if those assumptions hold. While some assumptions differ between types of test, others are common to all hypothesis tests.

Which of the following statements is a common assumption of hypothesis tests?

Possible Answers

- Sample observations are collected deterministically from the population.

- Sample observations are correlated with each other.

- <b><font color ='green'>Sample observations have no direct relationship with each other.</font></b>

- Sample sizes are greater than thirty observations.

All hypothesis tests assume that the data are collected at random from the population, that each row is independent of the others, and that the sample size is "big enough".